In [343]:
import pandas as pd
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import GridSearchCV

In [276]:
df = pd.read_csv('/train.csv',index_col='Id')
test = pd.read_csv('/test.csv',index_col = 'Id')

In [277]:
df.head()

,Home Ownership,Annual Income,Years in current job,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Bankruptcies,Purpose,Term,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score,Credit Default
Id,,,,,,,,,,,,,,,,,
0,Own Home,482087.0,NaN,0.0,11.0,26.3,685960.0,1.0,NaN,1.0,debt consolidation,Short Term,99999999.0,47386.0,7914.0,749.0,0
1,Own Home,1025487.0,10+ years,0.0,15.0,15.3,1181730.0,0.0,NaN,0.0,debt consolidation,Long Term,264968.0,394972.0,18373.0,737.0,1
2,Home Mortgage,751412.0,8 years,0.0,11.0,35.0,1182434.0,0.0,NaN,0.0,debt consolidation,Short Term,99999999.0,308389.0,13651.0,742.0,0
3,Own Home,805068.0,6 years,0.0,8.0,22.5,147400.0,1.0,NaN,1.0,debt consolidation,Short Term,121396.0,95855.0,11338.0,694.0,0
4,Rent,776264.0,8 years,0.0,13.0,13.6,385836.0,1.0,NaN,0.0,debt consolidation,Short Term,125840.0,93309.0,7180.0,719.0,0


In [278]:
df.columns = ['_'.join(col.split(' ')) for col in df.columns ]
df.columns = [col.lower() for col in df.columns]

test.columns = ['_'.join(col.split(' ')) for col in test.columns ]
test.columns = [col.lower() for col in test.columns]

In [279]:
df.head()

,home_ownership,annual_income,years_in_current_job,tax_liens,number_of_open_accounts,years_of_credit_history,maximum_open_credit,number_of_credit_problems,months_since_last_delinquent,bankruptcies,purpose,term,current_loan_amount,current_credit_balance,monthly_debt,credit_score,credit_default
Id,,,,,,,,,,,,,,,,,
0,Own Home,482087.0,NaN,0.0,11.0,26.3,685960.0,1.0,NaN,1.0,debt consolidation,Short Term,99999999.0,47386.0,7914.0,749.0,0
1,Own Home,1025487.0,10+ years,0.0,15.0,15.3,1181730.0,0.0,NaN,0.0,debt consolidation,Long Term,264968.0,394972.0,18373.0,737.0,1
2,Home Mortgage,751412.0,8 years,0.0,11.0,35.0,1182434.0,0.0,NaN,0.0,debt consolidation,Short Term,99999999.0,308389.0,13651.0,742.0,0
3,Own Home,805068.0,6 years,0.0,8.0,22.5,147400.0,1.0,NaN,1.0,debt consolidation,Short Term,121396.0,95855.0,11338.0,694.0,0
4,Rent,776264.0,8 years,0.0,13.0,13.6,385836.0,1.0,NaN,0.0,debt consolidation,Short Term,125840.0,93309.0,7180.0,719.0,0


In [280]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7500 entries, 0 to 7499
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   home_ownership                7500 non-null   object 
 1   annual_income                 5943 non-null   float64
 2   years_in_current_job          7129 non-null   object 
 3   tax_liens                     7500 non-null   float64
 4   number_of_open_accounts       7500 non-null   float64
 5   years_of_credit_history       7500 non-null   float64
 6   maximum_open_credit           7500 non-null   float64
 7   number_of_credit_problems     7500 non-null   float64
 8   months_since_last_delinquent  3419 non-null   float64
 9   bankruptcies                  7486 non-null   float64
 10  purpose                       7500 non-null   object 
 11  term                          7500 non-null   object 
 12  current_loan_amount           7500 non-null   float64
 13  cur

In [281]:
df.describe()

,annual_income,tax_liens,number_of_open_accounts,years_of_credit_history,maximum_open_credit,number_of_credit_problems,months_since_last_delinquent,bankruptcies,current_loan_amount,current_credit_balance,monthly_debt,credit_score,credit_default
count,5.943000e+03,7500.000000,7500.000000,7500.000000,7.500000e+03,7500.000000,3419.000000,7486.000000,7.500000e+03,7.500000e+03,7500.000000,5943.000000,7500.000000
mean,1.366392e+06,0.030133,11.130933,18.317467,9.451537e+05,0.170000,34.692600,0.117152,1.187318e+07,2.898332e+05,18314.454133,1151.087498,0.281733
std,8.453392e+05,0.271604,4.908924,7.041946,1.602622e+07,0.498598,21.688806,0.347192,3.192612e+07,3.178714e+05,11926.764673,1604.451418,0.449874
min,1.645970e+05,0.000000,2.000000,4.000000,0.000000e+00,0.000000,0.000000,0.000000,1.124200e+04,0.000000e+00,0.000000,585.000000,0.000000
25%,8.443410e+05,0.000000,8.000000,13.500000,2.792295e+05,0.000000,16.000000,0.000000,1.801690e+05,1.142565e+05,10067.500000,711.000000,0.000000
50%,1.168386e+06,0.000000,10.000000,17.000000,4.781590e+05,0.000000,32.000000,0.000000,3.095730e+05,2.093230e+05,16076.500000,731.000000,0.000000
75%,1.640137e+06,0.000000,14.000000,21.800000,7.935015e+05,0.000000,50.000000,0.000000,5.198820e+05,3.604062e+05,23818.000000,743.000000,1.000000
max,1.014934e+07,7.000000,43.000000,57.700000,1.304726e+09,7.000000,118.000000,4.000000,1.000000e+08,6.506797e+06,136679.000000,7510.000000,1.000000


In [282]:
#KEEPING A COPY BEFORE TRANSFORMATIONS
df1 = df.copy()

In [283]:
df.years_in_current_job.value_counts()

10+ years    2332
2 years       705
3 years       620
< 1 year      563
5 years       516
1 year        504
4 years       469
6 years       426
7 years       396
8 years       339
9 years       259
Name: years_in_current_job, dtype: int64

In [284]:
df.years_in_current_job.value_counts().index

Index(['10+ years', '2 years', '3 years', '< 1 year', '5 years', '1 year',
       '4 years', '6 years', '7 years', '8 years', '9 years'],
      dtype='object')

In [285]:
def years_cj(s):
  if s == '10+ years':
    return 10
  elif s ==  '2 years':
    return 2
  elif s == '3 years':
    return 3
  elif s == '< 1 year':
    return 0
  elif s == '5 years':
    return 5
  elif s == '1 year':
    return 1
  elif s == '4 years':
    return 4
  elif s == '6 years':
    return 6
  elif s=='7 years':
    return 7
  elif s== '8 years':
    return 8
  elif s == '9 years':
    return 9
  else:
    return -1

In [286]:
df.years_in_current_job.map(years_cj)
test.years_in_current_job.map(years_cj)

Id
7500     4
7501     1
7502     3
7503    10
7504     6
        ..
9995    10
9996     2
9997     2
9998    10
9999    10
Name: years_in_current_job, Length: 2500, dtype: int64

In [287]:
df.drop('years_in_current_job',axis = 1, inplace = True)
test.drop('years_in_current_job',axis = 1, inplace = True)

In [288]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7500 entries, 0 to 7499
Data columns (total 16 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   home_ownership                7500 non-null   object 
 1   annual_income                 5943 non-null   float64
 2   tax_liens                     7500 non-null   float64
 3   number_of_open_accounts       7500 non-null   float64
 4   years_of_credit_history       7500 non-null   float64
 5   maximum_open_credit           7500 non-null   float64
 6   number_of_credit_problems     7500 non-null   float64
 7   months_since_last_delinquent  3419 non-null   float64
 8   bankruptcies                  7486 non-null   float64
 9   purpose                       7500 non-null   object 
 10  term                          7500 non-null   object 
 11  current_loan_amount           7500 non-null   float64
 12  current_credit_balance        7500 non-null   float64
 13  mon

In [289]:
df.nunique()

home_ownership                     4
annual_income                   5478
tax_liens                          8
number_of_open_accounts           39
years_of_credit_history          408
maximum_open_credit             6963
number_of_credit_problems          8
months_since_last_delinquent      89
bankruptcies                       5
purpose                           15
term                               2
current_loan_amount             5386
current_credit_balance          6592
monthly_debt                    6716
credit_score                     268
credit_default                     2
dtype: int64

In [290]:
df.home_ownership.value_counts()

Home Mortgage    3637
Rent             3204
Own Home          647
Have Mortgage      12
Name: home_ownership, dtype: int64

In [291]:
#since home_ownership has no null values we can use get_dummies in the default mode as we dont need a nan column
new_cols = pd.get_dummies(df['home_ownership'])
new_cols_test = pd.get_dummies(test['home_ownership'])

In [292]:
new_cols

,Have Mortgage,Home Mortgage,Own Home,Rent
Id,,,,
0,0,0,1,0
1,0,0,1,0
2,0,1,0,0
3,0,0,1,0
4,0,0,0,1
...,...,...,...,...
7495,0,0,0,1
7496,0,1,0,0
7497,0,0,0,1


In [293]:
for i in new_cols.columns:
  k = '_'.join(i.split(" ")).lower()
  df[k] = new_cols[i]

for i in new_cols_test.columns:
  k = '_'.join(i.split(" ")).lower()
  test[k] = new_cols_test[i]


In [294]:
df.head()

,home_ownership,annual_income,tax_liens,number_of_open_accounts,years_of_credit_history,maximum_open_credit,number_of_credit_problems,months_since_last_delinquent,bankruptcies,purpose,term,current_loan_amount,current_credit_balance,monthly_debt,credit_score,credit_default,have_mortgage,home_mortgage,own_home,rent
Id,,,,,,,,,,,,,,,,,,,,
0,Own Home,482087.0,0.0,11.0,26.3,685960.0,1.0,NaN,1.0,debt consolidation,Short Term,99999999.0,47386.0,7914.0,749.0,0,0,0,1,0
1,Own Home,1025487.0,0.0,15.0,15.3,1181730.0,0.0,NaN,0.0,debt consolidation,Long Term,264968.0,394972.0,18373.0,737.0,1,0,0,1,0
2,Home Mortgage,751412.0,0.0,11.0,35.0,1182434.0,0.0,NaN,0.0,debt consolidation,Short Term,99999999.0,308389.0,13651.0,742.0,0,0,1,0,0
3,Own Home,805068.0,0.0,8.0,22.5,147400.0,1.0,NaN,1.0,debt consolidation,Short Term,121396.0,95855.0,11338.0,694.0,0,0,0,1,0
4,Rent,776264.0,0.0,13.0,13.6,385836.0,1.0,NaN,0.0,debt consolidation,Short Term,125840.0,93309.0,7180.0,719.0,0,0,0,0,1


In [295]:
df.drop('home_ownership',inplace = True, axis = 'columns')

test.drop('home_ownership',inplace = True, axis = 'columns')

In [296]:
#now onto the purpose column

In [297]:
df.purpose.value_counts()

debt consolidation      5944
other                    665
home improvements        412
business loan            129
buy a car                 96
medical bills             71
major purchase            40
take a trip               37
buy house                 34
small business            26
wedding                   15
moving                    11
educational expenses      10
vacation                   8
renewable energy           2
Name: purpose, dtype: int64

In [298]:
new_cols = pd.get_dummies(df['purpose'])

new_cols_test = pd.get_dummies(test['purpose'])

In [299]:
for i in new_cols.columns:
  k = '_'.join(i.split(" ")).lower()
  df[k] = new_cols[i]

for i in new_cols_test.columns:
  k = '_'.join(i.split(" ")).lower()
  test[k] = new_cols_test[i]

In [300]:
df.drop('purpose',inplace = True, axis = 1)

test.drop('purpose',inplace = True, axis = 1)

In [301]:
df.head()

,annual_income,tax_liens,number_of_open_accounts,years_of_credit_history,maximum_open_credit,number_of_credit_problems,months_since_last_delinquent,bankruptcies,term,current_loan_amount,...,home_improvements,major_purchase,medical_bills,moving,other,renewable_energy,small_business,take_a_trip,vacation,wedding
Id,,,,,,,,,,,,,,,,,,,,,
0,482087.0,0.0,11.0,26.3,685960.0,1.0,NaN,1.0,Short Term,99999999.0,...,0,0,0,0,0,0,0,0,0,0
1,1025487.0,0.0,15.0,15.3,1181730.0,0.0,NaN,0.0,Long Term,264968.0,...,0,0,0,0,0,0,0,0,0,0
2,751412.0,0.0,11.0,35.0,1182434.0,0.0,NaN,0.0,Short Term,99999999.0,...,0,0,0,0,0,0,0,0,0,0
3,805068.0,0.0,8.0,22.5,147400.0,1.0,NaN,1.0,Short Term,121396.0,...,0,0,0,0,0,0,0,0,0,0
4,776264.0,0.0,13.0,13.6,385836.0,1.0,NaN,0.0,Short Term,125840.0,...,0,0,0,0,0,0,0,0,0,0


In [302]:
#onto term

df.term.value_counts()

Short Term    5556
Long Term     1944
Name: term, dtype: int64

In [303]:
new_cols = pd.get_dummies(df['term'])

new_cols_test = pd.get_dummies(test['term'])

In [304]:
for i in new_cols.columns:
  k = '_'.join(i.split(" ")).lower()
  df[k] = new_cols[i]

for i in new_cols_test.columns:
  k = '_'.join(i.split(" ")).lower()
  test[k] = new_cols_test[i]

In [305]:
df.drop('term',inplace = True,axis = 1)
test.drop('term',inplace = True,axis = 1)

In [308]:
df.info(),test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7500 entries, 0 to 7499
Data columns (total 34 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   annual_income                 5943 non-null   float64
 1   tax_liens                     7500 non-null   float64
 2   number_of_open_accounts       7500 non-null   float64
 3   years_of_credit_history       7500 non-null   float64
 4   maximum_open_credit           7500 non-null   float64
 5   number_of_credit_problems     7500 non-null   float64
 6   months_since_last_delinquent  3419 non-null   float64
 7   bankruptcies                  7486 non-null   float64
 8   current_loan_amount           7500 non-null   float64
 9   current_credit_balance        7500 non-null   float64
 10  monthly_debt                  7500 non-null   float64
 11  credit_score                  5943 non-null   float64
 12  credit_default                7500 non-null   int64  
 13  hav

(None, None)

In [309]:
df.credit_default

Id
0       0
1       1
2       0
3       0
4       0
       ..
7495    0
7496    1
7497    0
7498    0
7499    0
Name: credit_default, Length: 7500, dtype: int64

In [310]:
#keeping a copy
df2 = df.copy()

In [311]:
#find out what proportion are missing values
df.isna().sum() / df.shape[0]

annual_income                   0.207600
tax_liens                       0.000000
number_of_open_accounts         0.000000
years_of_credit_history         0.000000
maximum_open_credit             0.000000
number_of_credit_problems       0.000000
months_since_last_delinquent    0.544133
bankruptcies                    0.001867
current_loan_amount             0.000000
current_credit_balance          0.000000
monthly_debt                    0.000000
credit_score                    0.207600
credit_default                  0.000000
have_mortgage                   0.000000
home_mortgage                   0.000000
own_home                        0.000000
rent                            0.000000
business_loan                   0.000000
buy_a_car                       0.000000
buy_house                       0.000000
debt_consolidation              0.000000
educational_expenses            0.000000
home_improvements               0.000000
major_purchase                  0.000000
medical_bills   

In [312]:
#finding correlation
correlation = df.corr()
correlation.style.background_gradient(cmap='coolwarm')

,annual_income,tax_liens,number_of_open_accounts,years_of_credit_history,maximum_open_credit,number_of_credit_problems,months_since_last_delinquent,bankruptcies,current_loan_amount,current_credit_balance,monthly_debt,credit_score,credit_default,have_mortgage,home_mortgage,own_home,rent,business_loan,buy_a_car,buy_house,debt_consolidation,educational_expenses,home_improvements,major_purchase,medical_bills,moving,other,renewable_energy,small_business,take_a_trip,vacation,wedding,long_term,short_term
annual_income,1.000000,0.066245,0.188524,0.186531,0.062218,-0.015407,-0.101787,-0.066877,0.032082,0.386481,0.584819,-0.037832,-0.101375,-0.011860,0.223850,-0.070768,-0.185264,0.033769,-0.012843,0.030701,-0.045382,-0.008615,0.076814,-0.001622,0.027635,-0.017165,-0.021996,nan,-0.002733,-0.017034,0.007267,0.038794,0.096060,-0.096060
tax_liens,0.066245,1.000000,-0.005360,0.009109,-0.002410,0.595339,0.052446,0.057360,-0.007835,-0.010195,0.027211,0.010581,0.024368,-0.004442,-0.015319,0.004378,0.013351,0.019306,0.000468,0.007129,-0.015876,-0.004054,-0.003049,0.018838,0.029712,-0.004252,0.005114,-0.001812,-0.006544,-0.007812,-0.003626,-0.004967,0.004953,-0.004953
number_of_open_accounts,0.188524,-0.005360,1.000000,0.144819,0.023897,-0.017813,-0.032740,-0.019130,0.001346,0.267325,0.413904,0.016095,0.028884,0.003690,0.121464,-0.005681,-0.119790,-0.032360,-0.027927,-0.018378,0.081440,0.019124,-0.004404,-0.000834,-0.013548,-0.010250,-0.069763,-0.008754,0.002124,-0.035607,0.015771,-0.012138,0.071316,-0.071316
years_of_credit_history,0.186531,0.009109,0.144819,1.000000,0.029247,0.071259,-0.015375,0.085699,0.010623,0.217856,0.191978,-0.015683,-0.020567,0.011745,0.187605,0.017448,-0.200392,-0.019232,-0.020277,-0.006115,-0.005502,0.032653,0.041071,-0.015546,0.024181,-0.011921,-0.007881,0.016079,0.000466,-0.021714,-0.002807,-0.007104,0.044243,-0.044243
maximum_open_credit,0.062218,-0.002410,0.023897,0.029247,1.000000,-0.004750,-0.003849,-0.011650,-0.002334,0.088090,0.010197,-0.009136,-0.014275,0.077782,0.019018,-0.004348,-0.023030,-0.002103,-0.000661,-0.002021,-0.007237,-0.001033,0.001611,-0.001960,-0.002672,-0.001552,0.002771,-0.000720,-0.002066,0.042426,-0.001180,-0.000872,-0.006023,0.006023
number_of_credit_problems,-0.015407,0.595339,-0.017813,0.071259,-0.004750,1.000000,0.139905,0.730751,-0.008596,-0.124137,-0.042252,-0.018473,0.020088,0.013115,-0.009252,0.001915,0.007201,-0.003970,-0.007899,0.000876,-0.009550,-0.012459,0.021080,0.008078,0.008092,0.000908,-0.002869,-0.005569,-0.020111,0.002710,0.054400,-0.009278,-0.020433,0.020433
months_since_last_delinquent,-0.101787,0.052446,-0.032740,-0.015375,-0.003849,0.139905,1.000000,0.162710,0.000459,-0.027198,-0.057702,-0.004395,0.002598,-0.016067,-0.065129,-0.003851,0.069999,-0.004646,-0.012561,0.011932,0.025765,-0.035462,-0.014725,-0.014252,-0.005106,-0.005105,-0.010244,-0.007645,-0.001754,0.000286,0.017673,-0.004132,0.014016,-0.014016
bankruptcies,-0.066877,0.057360,-0.019130,0.085699,-0.011650,0.730751,0.162710,1.000000,-0.005987,-0.142849,-0.079579,-0.027488,0.001648,0.005715,-0.006812,-0.006101,0.009879,-0.029667,-0.007683,-0.005626,0.008884,-0.011037,0.034982,0.001657,-0.005231,0.007145,-0.014155,-0.005516,-0.019534,-0.001836,0.000739,-0.015120,-0.031367,0.031367
current_loan_amount,0.032082,-0.007835,0.001346,0.010623,-0.002334,-0.008596,0.000459,-0.005987,1.000000,0.016672,-0.009382,-0.108552,-0.226522,-0.014737,0.015325,0.004196,-0.016674,-0.012513,-0.000939,0.018975,-0.002849,0.020886,0.026016,-0.009494,-0.001277,0.018593,-0.017054,-0.006014,-0.000162,-0.008036,0.000746,0.011664,-0.075222,0.075222
current_credit_balance,0.386481,-0.010195,0.267325,0.217856,0.088090,-0.124137,-0.027198,-0.142849,0.016672,1.000000,0.504964,-0.006159,-0.019522,-0.007296,0.171672,0.007212,-0.176947,-0.020891,-0.012149,-0.034777,0.082263,-0.010850,0.001251,-0.021426,-0.020934,-0.027408,-0.064743,-0.008524,-0.023026,-0.027036,-0.012027,-0.000186,0.108561,-0.108561


In [313]:
#dropping number of credit problems

In [334]:
df.drop('number_of_credit_problems', axis=1, inplace=True)
test.drop('number_of_credit_problems', axis=1, inplace=True)

In [315]:
#repeat

In [316]:
correlation = df.corr()
correlation.style.background_gradient(cmap='coolwarm')

,annual_income,tax_liens,number_of_open_accounts,years_of_credit_history,maximum_open_credit,months_since_last_delinquent,bankruptcies,current_loan_amount,current_credit_balance,monthly_debt,credit_score,credit_default,have_mortgage,home_mortgage,own_home,rent,business_loan,buy_a_car,buy_house,debt_consolidation,educational_expenses,home_improvements,major_purchase,medical_bills,moving,other,renewable_energy,small_business,take_a_trip,vacation,wedding,long_term,short_term
annual_income,1.000000,0.066245,0.188524,0.186531,0.062218,-0.101787,-0.066877,0.032082,0.386481,0.584819,-0.037832,-0.101375,-0.011860,0.223850,-0.070768,-0.185264,0.033769,-0.012843,0.030701,-0.045382,-0.008615,0.076814,-0.001622,0.027635,-0.017165,-0.021996,nan,-0.002733,-0.017034,0.007267,0.038794,0.096060,-0.096060
tax_liens,0.066245,1.000000,-0.005360,0.009109,-0.002410,0.052446,0.057360,-0.007835,-0.010195,0.027211,0.010581,0.024368,-0.004442,-0.015319,0.004378,0.013351,0.019306,0.000468,0.007129,-0.015876,-0.004054,-0.003049,0.018838,0.029712,-0.004252,0.005114,-0.001812,-0.006544,-0.007812,-0.003626,-0.004967,0.004953,-0.004953
number_of_open_accounts,0.188524,-0.005360,1.000000,0.144819,0.023897,-0.032740,-0.019130,0.001346,0.267325,0.413904,0.016095,0.028884,0.003690,0.121464,-0.005681,-0.119790,-0.032360,-0.027927,-0.018378,0.081440,0.019124,-0.004404,-0.000834,-0.013548,-0.010250,-0.069763,-0.008754,0.002124,-0.035607,0.015771,-0.012138,0.071316,-0.071316
years_of_credit_history,0.186531,0.009109,0.144819,1.000000,0.029247,-0.015375,0.085699,0.010623,0.217856,0.191978,-0.015683,-0.020567,0.011745,0.187605,0.017448,-0.200392,-0.019232,-0.020277,-0.006115,-0.005502,0.032653,0.041071,-0.015546,0.024181,-0.011921,-0.007881,0.016079,0.000466,-0.021714,-0.002807,-0.007104,0.044243,-0.044243
maximum_open_credit,0.062218,-0.002410,0.023897,0.029247,1.000000,-0.003849,-0.011650,-0.002334,0.088090,0.010197,-0.009136,-0.014275,0.077782,0.019018,-0.004348,-0.023030,-0.002103,-0.000661,-0.002021,-0.007237,-0.001033,0.001611,-0.001960,-0.002672,-0.001552,0.002771,-0.000720,-0.002066,0.042426,-0.001180,-0.000872,-0.006023,0.006023
months_since_last_delinquent,-0.101787,0.052446,-0.032740,-0.015375,-0.003849,1.000000,0.162710,0.000459,-0.027198,-0.057702,-0.004395,0.002598,-0.016067,-0.065129,-0.003851,0.069999,-0.004646,-0.012561,0.011932,0.025765,-0.035462,-0.014725,-0.014252,-0.005106,-0.005105,-0.010244,-0.007645,-0.001754,0.000286,0.017673,-0.004132,0.014016,-0.014016
bankruptcies,-0.066877,0.057360,-0.019130,0.085699,-0.011650,0.162710,1.000000,-0.005987,-0.142849,-0.079579,-0.027488,0.001648,0.005715,-0.006812,-0.006101,0.009879,-0.029667,-0.007683,-0.005626,0.008884,-0.011037,0.034982,0.001657,-0.005231,0.007145,-0.014155,-0.005516,-0.019534,-0.001836,0.000739,-0.015120,-0.031367,0.031367
current_loan_amount,0.032082,-0.007835,0.001346,0.010623,-0.002334,0.000459,-0.005987,1.000000,0.016672,-0.009382,-0.108552,-0.226522,-0.014737,0.015325,0.004196,-0.016674,-0.012513,-0.000939,0.018975,-0.002849,0.020886,0.026016,-0.009494,-0.001277,0.018593,-0.017054,-0.006014,-0.000162,-0.008036,0.000746,0.011664,-0.075222,0.075222
current_credit_balance,0.386481,-0.010195,0.267325,0.217856,0.088090,-0.027198,-0.142849,0.016672,1.000000,0.504964,-0.006159,-0.019522,-0.007296,0.171672,0.007212,-0.176947,-0.020891,-0.012149,-0.034777,0.082263,-0.010850,0.001251,-0.021426,-0.020934,-0.027408,-0.064743,-0.008524,-0.023026,-0.027036,-0.012027,-0.000186,0.108561,-0.108561
monthly_debt,0.584819,0.027211,0.413904,0.191978,0.010197,-0.057702,-0.079579,-0.009382,0.504964,1.000000,0.000234,0.018480,0.003482,0.209121,-0.039251,-0.189281,-0.021551,-0.044142,-0.017858,0.079160,-0.010085,0.001189,-0.003759,0.004669,-0.033818,-0.066045,-0.013920,-0.018991,-0.027146,0.001685,0.000943,0.144021,-0.144021


In [335]:
#remove monthly debt
df.drop('monthly_debt', axis=1, inplace=True)
test.drop('monthly_debt', axis=1, inplace=True)

In [336]:
#remove take a trip
df.drop('take_a_trip',axis = 1, inplace = True)
test.drop('take_a_trip',axis = 1, inplace = True)


In [319]:
correlation = df.corr()
correlation.style.background_gradient(cmap='coolwarm')

,annual_income,tax_liens,number_of_open_accounts,years_of_credit_history,maximum_open_credit,months_since_last_delinquent,bankruptcies,current_loan_amount,current_credit_balance,credit_score,credit_default,have_mortgage,home_mortgage,own_home,rent,business_loan,buy_a_car,buy_house,debt_consolidation,educational_expenses,home_improvements,major_purchase,medical_bills,moving,other,renewable_energy,small_business,vacation,wedding,long_term,short_term
annual_income,1.000000,0.066245,0.188524,0.186531,0.062218,-0.101787,-0.066877,0.032082,0.386481,-0.037832,-0.101375,-0.011860,0.223850,-0.070768,-0.185264,0.033769,-0.012843,0.030701,-0.045382,-0.008615,0.076814,-0.001622,0.027635,-0.017165,-0.021996,nan,-0.002733,0.007267,0.038794,0.096060,-0.096060
tax_liens,0.066245,1.000000,-0.005360,0.009109,-0.002410,0.052446,0.057360,-0.007835,-0.010195,0.010581,0.024368,-0.004442,-0.015319,0.004378,0.013351,0.019306,0.000468,0.007129,-0.015876,-0.004054,-0.003049,0.018838,0.029712,-0.004252,0.005114,-0.001812,-0.006544,-0.003626,-0.004967,0.004953,-0.004953
number_of_open_accounts,0.188524,-0.005360,1.000000,0.144819,0.023897,-0.032740,-0.019130,0.001346,0.267325,0.016095,0.028884,0.003690,0.121464,-0.005681,-0.119790,-0.032360,-0.027927,-0.018378,0.081440,0.019124,-0.004404,-0.000834,-0.013548,-0.010250,-0.069763,-0.008754,0.002124,0.015771,-0.012138,0.071316,-0.071316
years_of_credit_history,0.186531,0.009109,0.144819,1.000000,0.029247,-0.015375,0.085699,0.010623,0.217856,-0.015683,-0.020567,0.011745,0.187605,0.017448,-0.200392,-0.019232,-0.020277,-0.006115,-0.005502,0.032653,0.041071,-0.015546,0.024181,-0.011921,-0.007881,0.016079,0.000466,-0.002807,-0.007104,0.044243,-0.044243
maximum_open_credit,0.062218,-0.002410,0.023897,0.029247,1.000000,-0.003849,-0.011650,-0.002334,0.088090,-0.009136,-0.014275,0.077782,0.019018,-0.004348,-0.023030,-0.002103,-0.000661,-0.002021,-0.007237,-0.001033,0.001611,-0.001960,-0.002672,-0.001552,0.002771,-0.000720,-0.002066,-0.001180,-0.000872,-0.006023,0.006023
months_since_last_delinquent,-0.101787,0.052446,-0.032740,-0.015375,-0.003849,1.000000,0.162710,0.000459,-0.027198,-0.004395,0.002598,-0.016067,-0.065129,-0.003851,0.069999,-0.004646,-0.012561,0.011932,0.025765,-0.035462,-0.014725,-0.014252,-0.005106,-0.005105,-0.010244,-0.007645,-0.001754,0.017673,-0.004132,0.014016,-0.014016
bankruptcies,-0.066877,0.057360,-0.019130,0.085699,-0.011650,0.162710,1.000000,-0.005987,-0.142849,-0.027488,0.001648,0.005715,-0.006812,-0.006101,0.009879,-0.029667,-0.007683,-0.005626,0.008884,-0.011037,0.034982,0.001657,-0.005231,0.007145,-0.014155,-0.005516,-0.019534,0.000739,-0.015120,-0.031367,0.031367
current_loan_amount,0.032082,-0.007835,0.001346,0.010623,-0.002334,0.000459,-0.005987,1.000000,0.016672,-0.108552,-0.226522,-0.014737,0.015325,0.004196,-0.016674,-0.012513,-0.000939,0.018975,-0.002849,0.020886,0.026016,-0.009494,-0.001277,0.018593,-0.017054,-0.006014,-0.000162,0.000746,0.011664,-0.075222,0.075222
current_credit_balance,0.386481,-0.010195,0.267325,0.217856,0.088090,-0.027198,-0.142849,0.016672,1.000000,-0.006159,-0.019522,-0.007296,0.171672,0.007212,-0.176947,-0.020891,-0.012149,-0.034777,0.082263,-0.010850,0.001251,-0.021426,-0.020934,-0.027408,-0.064743,-0.008524,-0.023026,-0.012027,-0.000186,0.108561,-0.108561
credit_score,-0.037832,0.010581,0.016095,-0.015683,-0.009136,-0.004395,-0.027488,-0.108552,-0.006159,1.000000,0.442050,-0.010120,-0.029684,0.025985,0.016003,0.022816,0.004530,-0.008789,0.009890,-0.010300,-0.024201,-0.000073,0.000501,0.005489,-0.003473,nan,0.016501,-0.008246,-0.012231,0.044521,-0.044521


In [320]:
#no more strong correlations

In [321]:
df.columns

Index(['annual_income', 'tax_liens', 'number_of_open_accounts',
       'years_of_credit_history', 'maximum_open_credit',
       'months_since_last_delinquent', 'bankruptcies', 'current_loan_amount',
       'current_credit_balance', 'credit_score', 'credit_default',
       'have_mortgage', 'home_mortgage', 'own_home', 'rent', 'business_loan',
       'buy_a_car', 'buy_house', 'debt_consolidation', 'educational_expenses',
       'home_improvements', 'major_purchase', 'medical_bills', 'moving',
       'other', 'renewable_energy', 'small_business', 'vacation', 'wedding',
       'long_term', 'short_term'],
      dtype='object')

In [322]:
#fill in the gaps of features of type float64 with median values, and fill in the gaps "Years in current job" with mods
df['annual_income'].fillna(df['annual_income'].median(), inplace=True)
df['months_since_last_delinquent'].fillna(df['months_since_last_delinquent'].median(), inplace=True)
df['bankruptcies'].fillna(df['bankruptcies'].median(), inplace=True)
df['credit_score'].fillna(df['credit_score'].median(), inplace=True)

In [324]:
# fill in the gaps in the test data in the same way as the training data
test['annual_income'].fillna(test['annual_income'].median(), inplace=True)
test['months_since_last_delinquent'].fillna(test['months_since_last_delinquent'].median(), inplace=True)
test['bankruptcies'].fillna(test['bankruptcies'].median(), inplace=True)
test['credit_score'].fillna(test['credit_score'].median(), inplace=True)

In [323]:
#check again the number of gaps for each feature of the training dataset
df.isna().sum()

annual_income                   0
tax_liens                       0
number_of_open_accounts         0
years_of_credit_history         0
maximum_open_credit             0
months_since_last_delinquent    0
bankruptcies                    0
current_loan_amount             0
current_credit_balance          0
credit_score                    0
credit_default                  0
have_mortgage                   0
home_mortgage                   0
own_home                        0
rent                            0
business_loan                   0
buy_a_car                       0
buy_house                       0
debt_consolidation              0
educational_expenses            0
home_improvements               0
major_purchase                  0
medical_bills                   0
moving                          0
other                           0
renewable_energy                0
small_business                  0
vacation                        0
wedding                         0
long_term     

In [325]:
test.isna().sum()

annual_income                   0
tax_liens                       0
number_of_open_accounts         0
years_of_credit_history         0
maximum_open_credit             0
number_of_credit_problems       0
months_since_last_delinquent    0
bankruptcies                    0
current_loan_amount             0
current_credit_balance          0
monthly_debt                    0
credit_score                    0
have_mortgage                   0
home_mortgage                   0
own_home                        0
rent                            0
business_loan                   0
buy_a_car                       0
buy_house                       0
debt_consolidation              0
educational_expenses            0
home_improvements               0
major_purchase                  0
medical_bills                   0
moving                          0
other                           0
small_business                  0
take_a_trip                     0
vacation                        0
wedding       

In [329]:
set(df.columns.tolist()) - set(test.columns.tolist())

{'credit_default', 'renewable_energy'}

In [330]:
#rnewable enerfy doesnt appear in test and appears only twice in train so dropping
df.drop('renewable_energy',inplace = True,axis = 1)

In [331]:
#split the training dataset into X and y
X = df.drop('credit_default', axis=1)
y = df['credit_default']

In [337]:
# normalize the data
sc = MinMaxScaler()
train_std = sc.fit_transform(X)  # first we train
test_std = sc.transform(test)

In [342]:
#train a logistic regression model and look at the metrics
clf_log_reg = LogisticRegression().fit(train_std, y)
y_pred = clf_log_reg.predict(train_std)
y_proba = clf_log_reg.predict_proba(train_std)

print(f'ROC-AUC = {round(roc_auc_score(y, y_proba[:, 1]), 4)}\n')
print(classification_report(y, y_pred))

ROC-AUC = 0.7577

              precision    recall  f1-score   support

           0       0.77      0.99      0.86      5387
           1       0.88      0.24      0.37      2113

    accuracy                           0.78      7500
   macro avg       0.82      0.61      0.62      7500
weighted avg       0.80      0.78      0.73      7500



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [347]:
parameters = [{'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']},
              {'penalty':['none', 'elasticnet', 'l1', 'l2']},
              {'C':[0.001, 0.01, 0.1, 1, 10, 100]}]



grid_search = GridSearchCV(estimator = LogisticRegression(),
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 5,
                           verbose=0)


grid_search.fit(train_std, y)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid=[{'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                     'saga']},
                         {'penalty': ['none', 'elasticnet', 'l1', 'l2']},
                         {'C': [0.001, 0.01, 0.1, 1, 10, 100]}],
             scoring='accuracy')

In [348]:
grid_search.best_params_

{'solver': 'newton-cg'}